In [ ]:
!pip install nltk
!pip install numpy

In [ ]:
!pip install keras tensorflow

In [ ]:
import json
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import tensorflow as tf

Reading Data :

In [10]:
data_file = open('data.json').read()
intents = json.loads(data_file) 

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?' , '!', '.', ',', "'s", "'m", "'re", "'ll", "'ve", "'d", "'t"]

In [ ]:
import os
from nltk.data import find

try:
    find('tokenizers/punkt')
    print("Resource 'punkt' is installed and accessible.")
except LookupError:
    print("Resource 'punkt' not found.")


In [14]:
# Preprocess the file to accept the data (l' , tunisian dialect)
import re

def preprocess_text(text):
    # Replace accents and normalize text
    text = text.lower()
    text = re.sub(r'[^\w\s\'-]', '', text)  # Keep letters, numbers, spaces, apostrophes, and hyphens
    return text



In [ ]:
nltk.download('punkt_tab')

In [21]:
for intent in intents['root']['intents'] : 
    for pattern in intent['patterns'] :

        # tokenize each word in the sentence
        cleaned_pattern = preprocess_text(pattern)
        print(cleaned_pattern)
        w = nltk.word_tokenize(cleaned_pattern)
        words.extend(w)

        # add documents : 
        documents.append((w, intent['tag']))

        # add to our classes if it's a new label
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(words), "unique lemmatized words", words)
print(len(classes), "classes", classes)



hi
hello
good day
hey
how are you
bonjour
ahla
salam
asslema
bonsoir
salut
bye
ã plus
au revoir
prends soin de toi
ciao
saha
yalla ã  la prochaine
besslama
merci
merci beaucoup
je vous remercie
c'est gentil de votre part
je suis reconnaissant
chokran
barak allahou fik
comment obtenir une attestation de prãsence 
quels sont les documents nãcessaires pour lâattestation de prãsence 
oã¹ dois-je dãposer ma demande pour lâattestation de prãsence 
quelle est la procãdure pour avoir une attestation de prãsence 
combien de temps prend la dãlivrance de lâattestation de prãsence 
oã¹ puis-je trouver le formulaire pour lâattestation de prãsence 
est-ce que lâattestation de prãsence est prãªte en 48h 
dois-je signer le document avec deux professeurs 
puis-je dãposer ma demande en ligne pour lâattestation de prãsence 
est-ce que je dois fournir une copie de lâattestation dâinscription pour lâattestation de prãsence 
chedet houvour
kifash najem nakhou attestation de prãsence 
chnowa les papiers elli

In [29]:
# Initialize training data
training = []
output_empty = [0] * len(classes)

# Ensure bag length matches words and output_row matches classes
for doc in documents:
    # Initialize bag of words
    bag = []

    # List of tokenized words for the pattern
    pattern_words = doc[0]

    # Lemmatize each word and create a bag of words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Create bag of words with a fixed size equal to len(words)
    bag = [1 if w in pattern_words else 0 for w in words]

    # Create output_row with a fixed size equal to len(classes)
    output_row = [0] * len(classes)
    output_row[classes.index(doc[1])] = 1

    # Append to training data
    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Convert to a NumPy array after ensuring uniform size
training = np.array(training, dtype=object)

# Split into train_x and train_y
train_x = np.array([item[0] for item in training], dtype=np.float32)
train_y = np.array([item[1] for item in training], dtype=np.float32)

print("Training data created")


Training data created


In [31]:
# Create the model : 3 Layers , First layer 128 neurons, second layer 64 neurons, third layer 32 neurons

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compilation de modèle : 
sgd = SGD(learning_rate = 0.01 , momentum=0.9 , nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


c:\Users\MSI\Desktop\ATIA_LunarHack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
#fitting the model and saving it : 

hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model created")

Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1114 - loss: 2.4761
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4502 - loss: 1.8426
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6826 - loss: 1.1347
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7753 - loss: 0.7875
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8001 - loss: 0.6301
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8111 - loss: 0.5564
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8981 - loss: 0.3860
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8658 - loss: 0.4081
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9315 - loss: 0.2675
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9403 - loss: 0.2807
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9428 - loss: 0.2234
Epoch 12/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9694 - lo

Model created


Chatbot Prediction :

In [33]:
#méthode pour prédire la classe de l'input :
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [34]:
def bow(sentence , words , show_details=True):

    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)

    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

In [35]:
def predict_class(sentence , model):
    p = bow(sentence , words , show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25

    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [36]:
def getResponse(ints , intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['root']['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            response = random.choice(i['responses'])
            break
    return response

In [37]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [50]:
chatbot_response("Chokran") # test the chatbot response with a sample message

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


'De rien! Je suis lÃ\xa0 pour vous aider.'